In [ ]:
!pip install flask

In [ ]:
# from flask import Flask, request, render_template
# import torch
# from PIL import Image
# import os
# import io

# app = Flask(__name__)

# # Load the trained YOLOv5 model
# model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/best.pt')

# # Homepage route
# @app.route('/')
# def home():
#     return render_template('index.html')

# # Prediction route
# @app.route('/predict', methods=['POST'])
# def predict():
#     if 'file' not in request.files:
#         return "No file uploaded", 400

#     file = request.files['file']
#     if file.filename == '':
#         return "No file selected", 400

#     # Convert uploaded image to a PIL image
#     img_bytes = file.read()
#     img = Image.open(io.BytesIO(img_bytes))

#     # Run inference on the image
#     results = model(img)

#     # Return the model's prediction
#     return results.pandas().xyxy[0].to_json(orient="records")

# if __name__ == '__main__':
#     app.run()


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import torch
from PIL import Image
import io

app = Flask(__name__)
run_with_ngrok(app)  # Starts ngrok when the app runs

# Load the trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')  # Update the path if needed

# Define homepage route
@app.route('/')
def home():
    return '''<h1>Accident Severity Classifier</h1>
              <form method="POST" action="/predict" enctype="multipart/form-data">
              <input type="file" name="file">
              <input type="submit">
              </form>'''

# Define prediction route
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No file selected'}), 400

    # Convert uploaded image to PIL format
    img_bytes = file.read()
    img = Image.open(io.BytesIO(img_bytes))

    # Run the YOLOv5 model on the uploaded image
    results = model(img)

    # Convert results to a more readable format (pandas DataFrame)
    predictions = results.pandas().xyxy[0].to_json(orient="records")

    return predictions  # Return predictions as JSON

# Run the Flask app
if __name__ == '__main__':
  app.run()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-3 Python-3.10.12 torch-2.4.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/serving.py", line 757, in __init__
    self.server_bind()
  File "/usr/lib/python3.10/http/server.py", line 137, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.10/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-52-a8e5e2032ef8>", line 46, in <cell line: 45>
    app.run()
  File "/usr/local/lib/python3.10/dist-packages/flask_ngrok.py", line 88, in new_run
    old_run()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1191, in run
    run_simple(t.cast(str, host), port, self, **op

TypeError: object of type 'NoneType' has no len()

In [ ]:
from flask import Flask, request, render_template
import torch
from PIL import Image
import io
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # This will expose the app using ngrok

# Load your fine-tuned YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')

# Home route to display the upload form
@app.route('/')
def home():
    return render_template('index.html')

# Prediction route that handles image upload and model inference
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return "No file part", 400

    file = request.files['file']

    if file.filename == '':
        return "No image selected for uploading", 400

    # Load the image and run YOLOv5 inference
    img_bytes = file.read()
    img = Image.open(io.BytesIO(img_bytes))

    # Run inference
    results = model(img)

    # Get the results as a JSON response
    return results.pandas().xyxy[0].to_json(orient="records")

if __name__ == '__main__':
    app.run()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-3 Python-3.10.12 torch-2.4.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import os
import threading

from flask import Flask
from pyngrok import ngrok

app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
@app.route("/")
def index():
    return "Hello from Colab!"

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "https://9f48-35-237-253-65.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [ ]:
# !pip install flask-ngrok pyngrok

import torch
from flask import Flask, request, render_template
from PIL import Image
import io
from pyngrok import ngrok
import threading

app = Flask(__name__)

# Load the trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/best.pt')

# Homepage route
@app.route('/')
def home():
    return '''
    <!doctype html>
    <title>Accident Classification</title>
    <h1>Upload an image for classification (Severe/Moderate)</h1>
    <form method="POST" action="/predict" enctype="multipart/form-data">
      <input type="file" name="file">
      <input type="submit" value="Upload">
    </form>
    '''

# Prediction route
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return "No file uploaded", 400

    file = request.files['file']
    if file.filename == '':
        return "No file selected", 400

    # Convert uploaded image to a PIL image
    img_bytes = file.read()
    img = Image.open(io.BytesIO(img_bytes))

    # Run inference on the image
    results = model(img)
    # print(type(results.pandas().xyxy[0].to_json(orient="records")))
    # Return the model's prediction
    # results_df = results.pandas().xyxy[0]
    return results.pandas().xyxy[0]["name"].to_json(orient="records")

# Function to run Flask and ngrok in parallel
def run():

    # Open an ngrok tunnel to the Flask server
    public_url = ngrok.connect(4528).public_url
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
    # Update any base URLs to use the public ngrok URL
    app.config["BASE_URL"] = public_url
    # Start Flask app in a new thread
    threading.Thread(target=app.run, kwargs={"use_reloader": False, "port": 4528}).start()

# Run the application
run()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-3 Python-3.10.12 torch-2.4.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


 * ngrok tunnel "https://52dc-35-237-253-65.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:4528
INFO:werkzeug:Press CTRL+C to quit
